In [3]:
%%writefile saxpy_functor.cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/random.h>
#include <iostream>

struct saxpy
{
    int a;
    saxpy(int a_) : a(a_) {};  // Construtor que inicializa a constante 'a'

    __host__ __device__
    double operator()(const int& x, const int& y) const {
        return a * x + y;  // Operação que será aplicada a cada elemento de x e y
    }
};

int main() {
    const int N = 1000;  // Tamanho dos vetores
    thrust::device_vector<int> d_a(N);
    thrust::device_vector<int> d_b(N);
    thrust::device_vector<int> d_c(N);

    // Inicializando vetores aleatórios
    thrust::default_random_engine rng;
    thrust::uniform_int_distribution<int> dist(0, 100);

    for (int i = 0; i < N; ++i) {
        d_a[i] = dist(rng);
        d_b[i] = dist(rng);
    }

    int scalar = 5;  // Constante a para o cálculo Saxpy
    thrust::transform(d_a.begin(), d_a.end(), d_b.begin(), d_c.begin(), saxpy(scalar));

    // Exibir o resultado
    for (int i = 0; i < 10; ++i) {
        std::cout << "Resultado " << i << ": " << d_c[i] << std::endl;
    }

    return 0;
}

Writing saxpy_functor.cu


In [5]:
!nvcc -arch=sm_75 -std=c++14 saxpy_functor.cu -o saxpy_functor

In [6]:
!./saxpy_functor

Resultado 0: 8
Resultado 1: 390
Resultado 2: 504
Resultado 3: 300
Resultado 4: 205
Resultado 5: 101
Resultado 6: 371
Resultado 7: 346
Resultado 8: 540
Resultado 9: 462


In [11]:
%%writefile magnitude_reduce.cu
#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/host_vector.h>
#include <thrust/copy.h>
#include <cmath>
#include <iostream>

struct square
{
    __host__ __device__
    float operator()(const float& x) const {
        return x * x;
    }
};

float magnitude(const thrust::device_vector<float>& v) {
    float sum_of_squares = thrust::transform_reduce(v.begin(), v.end(), square(), 0.0f, thrust::plus<float>());
    return std::sqrt(sum_of_squares);
}

int main() {
    const int N = 1000;
    thrust::host_vector<float> h_v(N);

    // Inicializando o vetor com valores aleatórios
    for (int i = 0; i < N; ++i) {
        h_v[i] = static_cast<float>(rand()) / (RAND_MAX / 100);  // Usando rand() para gerar valores entre 0 e 100
    }

    // Copiando para a GPU
    thrust::device_vector<float> d_v = h_v;

    float mag = magnitude(d_v);
    std::cout << "Magnitude do vetor: " << mag << std::endl;

    return 0;
}

Overwriting magnitude_reduce.cu


In [12]:
!nvcc -arch=sm_75 -std=c++14 magnitude_reduce.cu -o magnitude_reduce

In [13]:
!./magnitude_reduce

Magnitude do vetor: 1840.52


In [14]:
%%writefile variance_fusion_kernel.cu
#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/host_vector.h>
#include <thrust/copy.h>
#include <iostream>
#include <cmath>

struct variance_op
{
    float mean;
    variance_op(float mean_) : mean(mean_) {}

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;
        return diff * diff;
    }
};

float calculate_mean(const thrust::device_vector<float>& d_vec) {
    float sum = thrust::reduce(d_vec.begin(), d_vec.end(), 0.0f, thrust::plus<float>());
    return sum / d_vec.size();
}

float calculate_variance(const thrust::device_vector<float>& d_vec, float mean) {
    return thrust::transform_reduce(d_vec.begin(), d_vec.end(), variance_op(mean), 0.0f, thrust::plus<float>()) / d_vec.size();
}

int main() {
    const int N = 1000;
    thrust::host_vector<float> h_vec(N);

    // Inicializando o vetor com valores aleatórios
    for (int i = 0; i < N; ++i) {
        h_vec[i] = static_cast<float>(rand()) / (RAND_MAX / 100);  // Usando rand() para gerar valores entre 0 e 100
    }

    // Copiando para a GPU
    thrust::device_vector<float> d_vec = h_vec;

    float mean = calculate_mean(d_vec);
    float variance = calculate_variance(d_vec, mean);

    std::cout << "Média: " << mean << std::endl;
    std::cout << "Variância: " << variance << std::endl;

    return 0;
}

Overwriting variance_fusion_kernel.cu


In [15]:
!nvcc -arch=sm_75 -std=c++14 variance_fusion_kernel.cu -o variance_fusion_kernel

In [16]:
!./variance_fusion_kernel

Média: 50.8125
Variância: 805.611
